Source : [Spatial Visualizations and Analysis in Python with Folium](http://https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf)

In [ ]:
! pip install -q folium

In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

In [ ]:
df_train = pd.read_csv('../input/nyc-taxi-trip-duration/train.zip').drop(columns=['trip_duration', 'dropoff_datetime'])
df_test = pd.read_csv('../input/nyc-taxi-trip-duration/test.zip')
df = pd.concat([df_train, df_test], sort=False, ignore_index=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.pickup_datetime = pd.to_datetime(df.pickup_datetime, format='%Y-%m-%d %H:%M:%S')
df['month'] = df.pickup_datetime.apply(lambda x: x.month)
df['week'] = df.pickup_datetime.apply(lambda x: x.week)
df['day'] = df.pickup_datetime.apply(lambda x: x.day)
df['hour'] = df.pickup_datetime.apply(lambda x: x.hour)

In [ ]:
df.head()

In [ ]:
def generateBaseMap(default_location=[40.693943, -73.985880], default_zoom_start=11):
    """
    location: Define the default location to zoom at when rendering the map
    zoom_start: The zoom level that the map will default to when rendering the map
    control_scale: Shows the map scale for a given zoom level
    """
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [ ]:
base_map = generateBaseMap()
base_map

**Analysis Question:**<br>
The city is planning to install taxi stops, locations throughout the city where people can get picked up/dropped off by cabs and wait for cabs to pick them up. This initiative is aiming to:
1. Reduce the amount of taxis that stops at the bicycle lanes 
2. Making it easier and safer for the citizens of the city to get a cab especially during night time 
3. Allowing taxi cab drivers to know where to go when they want to find riders
4. Allowing potential riders to have a point where they can go to when they are looking for taxis.

You are tasked with suggesting the best locations for these taxi stops, and also generating insights about the pattern of rides amount throughout the day across the city.

In [ ]:
# We will be using 2 months worth of data for this analysis, May - June 2016
df_copy = df[df.month>4].copy()
df_copy['count'] = 1

In [ ]:
df_copy[['pickup_latitude', 'pickup_longitude', 'count']].groupby(['pickup_latitude', 'pickup_longitude']).sum().sort_values('count', ascending=False).head(10)

In [ ]:
# From the dataframe above, it can be clearly seen that using tables to try to understand spatial data is an impossible task (duh)
base_map = generateBaseMap()
HeatMap(data=df_copy[['pickup_latitude', 'pickup_longitude', 'count']].groupby(['pickup_latitude', 'pickup_longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)

In [ ]:
base_map

In [ ]:
base_map.add_child(folium.ClickForMarker(popup='Potential Location'))

In [ ]:
df_hour_list = []
for hour in df_copy.hour.sort_values().unique():
    df_hour_list.append(df_copy.loc[df_copy.hour == hour, ['pickup_latitude', 'pickup_longitude', 'count']].groupby(['pickup_latitude', 'pickup_longitude']).sum().reset_index().values.tolist())

In [ ]:
base_map = generateBaseMap(default_zoom_start=11)
HeatMapWithTime(df_hour_list, radius=5, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map)
base_map